In [17]:
import requests

# Read the API key from the file
with open('census-data-api-key.txt', 'r') as file:
    api_key = file.read().strip()

# Construct the most basic API query URL for PUMS data
url = f'https://api.census.gov/data/2021/pums?get=SERIALNO,NP,BDSP&for=state:49&key={api_key}'

# Make the API request
response = requests.get(url)

# Check the response status and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.content[:500]}")  # Print only the first 500 characters


Status Code: 404
Response Content: b'<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>'


In [11]:
import requests

# Read the API key from the file
with open('census-data-api-key.txt', 'r') as file:
    api_key = file.read().strip()

# Construct the basic API query URL
url = f'https://api.census.gov/data/2021/acs/acs1?get=NAME,B01003_001E&for=us:*&key={api_key}'

# Make the API request
response = requests.get(url)

# Check the response status and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.content}")


Status Code: 200
Response Content: b'[["NAME","B01003_001E","us"],\n["United States","331893745","1"]]'


In [12]:
import requests
import pandas as pd

# Read the API key from the file
with open('census-data-api-key.txt', 'r') as file:
    api_key = file.read().strip()

# Construct the API query URL for PUMS data for Utah
#url = f'https://api.census.gov/data/2019/pums?get=SERIALNO,NP,BDSP&for=state:49&key={api_key}'
url = f'https://api.census.gov/data/2022/acs/acs1/pums?get=SERIALNO,NP,BDSP&for=state:49&key={api_key}'

# Make the API request
response = requests.get(url)

# Check the response status and content
if response.status_code == 200:
    # Convert the response to a Pandas DataFrame
    data = response.json()
    columns = data[0]
    rows = data[1:]
    pums_data = pd.DataFrame(rows, columns=columns)
    
    # Convert columns to appropriate data types
    pums_data['NP'] = pd.to_numeric(pums_data['NP'])
    pums_data['BDSP'] = pd.to_numeric(pums_data['BDSP'])
    
    # Display the first few rows of the downloaded data
    print(pums_data.head())
    
    # Filter the data to include only relevant columns
    filtered_data = pums_data[['BDSP', 'NP']]

    # Drop rows with missing values
    filtered_data = filtered_data.dropna()

    # Calculate the average household size based on the number of bedrooms
    average_household_size = filtered_data.groupby('BDSP')['NP'].mean().reset_index()

    # Rename columns for clarity
    average_household_size.columns = ['Number of Bedrooms', 'Average Household Size']

    # Display the results
    display(average_household_size)

else:
    print(f"Error: {response.status_code} - {response.content}")


        SERIALNO  NP  BDSP state
0  2022GQ0000032   1    -1    49
1  2022GQ0000163   1    -1    49
2  2022GQ0000198   1    -1    49
3  2022GQ0000320   1    -1    49
4  2022GQ0000483   1    -1    49


,Number of Bedrooms,Average Household Size
0,-1,1.000000
1,0,1.251429
2,1,1.300578
3,2,1.898191
4,3,2.465169
5,4,3.092894
6,5,3.668295
7,6,4.067039
8,8,4.060606


In [17]:
url = f'https://api.census.gov/data/2022/acs/acs1/pums?get=SERIALNO,NP,BDSP,BLD&for=state:49&key={api_key}'

# Make the API request
response = requests.get(url)

# Check the response status and content
if response.status_code == 200:
    # Convert the response to a Pandas DataFrame
    data = response.json()
    columns = data[0]
    rows = data[1:]
    pums_data = pd.DataFrame(rows, columns=columns)
    
    # Convert columns to appropriate data types
    pums_data['NP'] = pd.to_numeric(pums_data['NP'])
    pums_data['BDSP'] = pd.to_numeric(pums_data['BDSP'])
    pums_data['BLD'] = pd.to_numeric(pums_data['BLD'])
    
    # Display the first few rows of the downloaded data
    print(pums_data.head())
    
    # Filter the data to include only single-family and multi-family units
    single_family = pums_data[pums_data['BLD'].isin([2, 3])]
    multi_family = pums_data[pums_data['BLD'].isin([4, 5, 6, 7, 8, 9])]

    # Count the number of records
    single_family_count = len(single_family)
    multi_family_count = len(multi_family)

    # Analyze single-family data
    single_family_avg = single_family.groupby('BDSP')['NP'].agg(['mean', 'count']).reset_index()
    single_family_avg.columns = ['Number of Bedrooms', 'Average Household Size', 'Number of Records']

    # Analyze multi-family data
    multi_family_avg = multi_family.groupby('BDSP')['NP'].agg(['mean', 'count']).reset_index()
    multi_family_avg.columns = ['Number of Bedrooms', 'Average Household Size', 'Number of Records']

    # Display the results
    display(f"Single-Family Homes: {single_family_count} records")
    display(single_family_avg)
    
    display(f"Multi-Family Homes: {multi_family_count} records")
    display(multi_family_avg)

    # Optionally save the results to CSV files
    single_family_avg.to_csv('single_family_avg.csv', index=False)
    multi_family_avg.to_csv('multi_family_avg.csv', index=False)
else:
    print(f"Error: {response.status_code} - {response.content}")

        SERIALNO  NP  BDSP  BLD state
0  2022GQ0000032   1    -1    0    49
1  2022GQ0000163   1    -1    0    49
2  2022GQ0000198   1    -1    0    49
3  2022GQ0000320   1    -1    0    49
4  2022GQ0000483   1    -1    0    49


'Single-Family Homes: 9942 records'

,Number of Bedrooms,Average Household Size,Number of Records
0,0,1.566038,53
1,1,1.473282,131
2,2,1.792536,911
3,3,2.463816,3040
4,4,3.098708,2786
5,5,3.666503,2039
6,6,4.074157,890
7,8,4.282609,92


'Multi-Family Homes: 1973 records'

,Number of Bedrooms,Average Household Size,Number of Records
0,0,1.095652,115
1,1,1.250467,535
2,2,1.995604,910
3,3,2.562691,327
4,4,2.855072,69
5,5,4.125000,8
6,6,3.000000,3
7,8,1.000000,6
